# What is Ownership?

In [2]:
{
    let s = "hello";
} // s is no longer valid


()

In [3]:
let s = "hello";


In [4]:
:t s

s: &str

In [5]:
let s = String::from("hello");


In [6]:
:t s

s: String

In [7]:
let mut s = String::from("hello");
s.push_str(", world!"); // mutate
println!("{s}");


hello, world!


## `Copy`

In [8]:
{
    let s = String::from("hello");
} // s is no longer valid


()

In [9]:
let x = 5;
let y = x; // copy: stack only data


## move

In [10]:
// String: ptr, length, capacity
let s1 = String::from("hello");
let s2 = s1; // move:
// shallow copy: the ptr, length, capacity

// cannot use s1 again
println!("{s1}, world!");


Error: borrow of moved value: `s1`

## `Clone`

In [11]:
let s1 = String::from("hello");
let s2 = s1.clone(); // clone
println!("s1={s1}, s2={s2}");


s1=hello, s2=hello


## Ownership and Functions

In [12]:
fn takes_ownership(some_string: String) {
    println!("{some_string}");
}

fn makes_copy(some_integer: i32) {
    println!("{some_integer}");
}

{
    let s = String::from("hello");
    takes_ownership(s);

    let x = 5;
    makes_copy(x);
}


hello
5


()

## Return Values and Scope

In [13]:
fn gives_ownership() -> String {
    let some_string = String::from("yours");

    some_string
}

fn takes_and_gives_back(a_string: String) -> String {
    a_string
}

{
    let s1 = gives_ownership();

    let s2 = String::from("hello");
    let s3 = takes_and_gives_back(s2);
}


()

In [14]:
fn calculate_length(s: String) -> (String, usize) {
    let length = s.len();

    (s, length)
}

{
    let s1 = String::from("hello");
    let (s2, len) = calculate_length(s1);

    println!("The length of '{s2}' is {len}.");
}


The length of 'hello' is 5.


()

# References and Borrowing

In [15]:
// reference parameter
fn calculate_length(s: &String) -> usize {
    s.len()
}

{
    let s1 = String::from("hello");
    let len = calculate_length(&s1); // reference argument

    println!("The length of '{s1}' is {len}");
}


The length of 'hello' is 5


()

In [16]:
fn change(some_string: &String) {
    some_string.push_str(", world!"); // modify borrowed value
}

{
    let s = String::from("hello");
    change(&s);
}


Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

## Mutable References

In [17]:
// mutable reference parameter
fn change(some_string: &mut String) {
    some_string.push_str(", world!"); // modify borrowed value
}

{
    let mut s = String::from("hello"); // mutable variable
    change(&mut s); // mutable reference argument
    println!("{s}");
}


hello, world!


()

In [18]:
{
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;

    println!("{}, {}", r1, r2);
}


Error: cannot borrow `s` as mutable more than once at a time

In [19]:
{
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
        println!("{}", *r1);
    } // r1 goes out of scope
    let r2 = &mut s; // not simultaneous
    println!("{}", r2); // auto deference
}


hello
hello


()

In [20]:
// combine mutable and immutable references
{
    let mut s = String::from("hello");

    let r1 = &s;
    let r2 = &s;
    let r3 = &mut s;

    println!("{}, {}, {}", r1, r2, r3);
}


Error: cannot borrow `s` as mutable because it is also borrowed as immutable

In [21]:
{
    let mut s = String::from("hello");
    let r1 = &s;
    let r2 = &s;
    println!("{}, {}", r1, r2); // r1, r2 moved

    let r3 = &mut s;
    println!("{}", r3);
}


hello, hello
hello


()

## Dangling References

In [22]:
fn dangle() -> &String {
    let s = String::from("hello");

    &s
}

{
    let _reference_to_nothing = dangle();
}


Error: missing lifetime specifier

Error: cannot return reference to local variable `s`

In [23]:
// fix: move
fn no_dangle() -> String {
    let s = String::from("hello");

    s
}

{
    let _ = no_dangle();
}


()

# The Slice Type

In [24]:
fn first_word(s: &String) -> usize {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return i;
        }
    }

    s.len()
}

{
    let mut s = String::from("hello world");
    let word = first_word(&s);
    println!("{word}");

    s.clear();
}


5


()

## String Slices

In [25]:
{
    let s = String::from("hello world");
    let hello = &s[0..5];
    let world = &s[6..11];
    println!("{hello} {world}");
}


hello world


()

In [26]:
{
    let s = String::from("hello");

    let slice = &s[0..2];
    println!("{slice}");
    let slice = &s[..2]; // left: default 0
    println!("{slice}");

    let len = s.len();

    let slice = &s[3..len];
    println!("{slice}");
    let slice = &s[3..]; // right: default len-1
    println!("{slice}");

    let slice = &s[0..len];
    println!("{slice}");
    let slice = &s[..];
    println!("{slice}");
}


he
he
lo
lo
hello
hello


()

In [27]:
// &str: string slice
fn first_word(s: &String) -> &str {
    let bytes = s.as_bytes();
    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[..i];
        }
    }

    &s[..]
}

{
    let s = String::from("hello world");
    let fw = first_word(&s);
    println!("{fw}");
}


hello


()

In [28]:
{
    let mut s = String::from("hello world");

    let fw = first_word(&s);
    s.clear();

    println!("{fw}");
}


Error: cannot borrow `s` as mutable because it is also borrowed as immutable

In [29]:
// string literal: immutable
let s = "hello world";

In [30]:
:t s

s: &str

In [31]:
fn first_word(s: &str) -> &str {
  let bytes = s.as_bytes();
  for (i, &item) in bytes.iter().enumerate() {
      if item == b' ' {
          return &s[..i];
      }
  }

  &s[..]
}

{
  let s = String::from("hello world");
  let fw = first_word(&s[0..6]);
  let fw = first_word(&s);

  let sl = "hello world";
  let fw = first_word(&sl[0..6]);
  let fw = first_word(sl); // string literals are string slices
}

()

## Other Slices

In [32]:
{
    let a = [1, 2, 3, 4, 5];
    let slice = &a[1..3]; // &[i32]

    assert_eq!(slice, &[2, 3]);
}


()